In [ ]:
!pip install tldextract

In [ ]:
import pandas as pd
from tldextract import extract
from google.colab import drive
drive.mount('/content/drive')

**Malicious URLs**

In [ ]:
malicious_df = pd.read_csv("drive/My Drive/Colab Notebooks/Phishing_URL/verified_online.csv")
malicious_df.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6471412,https://www.accesodigitalfirma.com//bancos.php,http://www.phishtank.com/phish_detail.php?phis...,2020-03-27T02:32:06+00:00,yes,2020-03-27T02:34:33+00:00,yes,Other
1,6471402,https://iticadregisterbonus.ga/,http://www.phishtank.com/phish_detail.php?phis...,2020-03-27T02:00:23+00:00,yes,2020-03-27T02:01:46+00:00,yes,Other
2,6471389,https://www.mibcp.idlemovil.com/iniciar-sesion,http://www.phishtank.com/phish_detail.php?phis...,2020-03-27T01:45:10+00:00,yes,2020-03-27T01:45:24+00:00,yes,Other
3,6471388,https://jhtys.net/index/login/index.html,http://www.phishtank.com/phish_detail.php?phis...,2020-03-27T01:39:35+00:00,yes,2020-03-27T01:40:24+00:00,yes,Other
4,6471386,https://sicurezza-covid19.com/,http://www.phishtank.com/phish_detail.php?phis...,2020-03-27T01:36:00+00:00,yes,2020-03-27T01:39:08+00:00,yes,Other


In [ ]:
new_mal = malicious_df.drop(malicious_df.columns[[0, 2, 3, 4, 5, 6, 7]], axis=1) 
new_mal['label'] = 1
new_mal.head()

,url,label
0,https://www.accesodigitalfirma.com//bancos.php,1
1,https://iticadregisterbonus.ga/,1
2,https://www.mibcp.idlemovil.com/iniciar-sesion,1
3,https://jhtys.net/index/login/index.html,1
4,https://sicurezza-covid19.com/,1


In [ ]:
print( str(new_mal.shape[0]) + " phishing URLs")

13098 phishing URLs


**Handling Benign URLs**

In [ ]:
benign_df = pd.read_csv("drive/My Drive/Colab Notebooks/Phishing_URL/top-1m.csv", names=["url"])
benign_df['label'] = 0
benign_df.head()

,url,label
0,google.com,0
1,microsoft.com,0
2,www.google.com,0
3,windowsupdate.com,0
4,netflix.com,0


In [ ]:
new_ben = benign_df.sample(new_mal.shape[0])
print( str(new_ben.shape[0]) + " normal URLs")

13098 normal URLs


In [ ]:
dataframes = [new_mal, new_ben]
dataset = pd.concat(dataframes)
dataset.tail()

,url,label
945097,depkominfo.go.id,0
396710,realdmp.realclick.co.kr,0
607968,storage-ec2-924.sharefile.com,0
215241,autoimg.cn.ccgslb.com.cn,0
825069,www.hoopsrumors.com,0


**Extract the TLD**

In [ ]:
dataset.url = dataset.url.map(lambda u : extract(u).domain)
dataset.head()

,url,label
0,accesodigitalfirma,1
1,iticadregisterbonus,1
2,idlemovil,1
3,jhtys,1
4,sicurezza-covid19,1


In [ ]:
domain_char_list = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",
                    "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", 
                    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
                    "-", "_", "."]
len(domain_char_list)


65

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
dataset.shape[0]

26196

In [ ]:
# Filtering out IP addresses
dataset = dataset[~dataset['url'].str.contains("\\.")]
dataset.shape[0]

25888

In [ ]:
dataset['url'] = dataset['url'].map(lambda u : [domain_char_list.index(c) + 1 for c in u])
dataset

,url,label
0,"[1, 3, 3, 5, 19, 15, 4, 9, 7, 9, 20, 1, 12, 6,...",1
1,"[9, 20, 9, 3, 1, 4, 18, 5, 7, 9, 19, 20, 5, 18...",1
2,"[9, 4, 12, 5, 13, 15, 22, 9, 12]",1
3,"[10, 8, 20, 25, 19]",1
4,"[19, 9, 3, 21, 18, 5, 26, 26, 1, 63, 3, 15, 22...",1
...,...,...
945097,"[4, 5, 16, 11, 15, 13, 9, 14, 6, 15]",0
396710,"[18, 5, 1, 12, 3, 12, 9, 3, 11]",0
607968,"[19, 8, 1, 18, 5, 6, 9, 12, 5]",0
215241,"[3, 3, 7, 19, 12, 2]",0


In [ ]:
features = pad_sequences(dataset['url'], padding='post')
labels = dataset['label']

In [ ]:
features.shape
labels.head()

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model():
  model = Sequential()
  num_chars = len(domain_char_list)
  model.add(Embedding(input_dim=num_chars+1, output_dim=256, mask_zero=True))
  model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.0, return_sequences=False))
  model.add(Dense(units=1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
  return model

In [ ]:
from sklearn.model_selection import train_test_split
X, y = features, labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=28)

In [ ]:
model=build_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=16, batch_size=16)

Epoch 1/16
1084/1084 [==============================] - 140s 129ms/step - loss: 0.5801 - accuracy: 0.6924 - val_loss: 0.5725 - val_accuracy: 0.6898
Epoch 2/16
1084/1084 [==============================] - 139s 128ms/step - loss: 0.5476 - accuracy: 0.7131 - val_loss: 0.5533 - val_accuracy: 0.7072
Epoch 3/16
1084/1084 [==============================] - 139s 129ms/step - loss: 0.5284 - accuracy: 0.7238 - val_loss: 0.5357 - val_accuracy: 0.7230
Epoch 4/16
1084/1084 [==============================] - 139s 128ms/step - loss: 0.5025 - accuracy: 0.7451 - val_loss: 0.5132 - val_accuracy: 0.7416
Epoch 5/16
1084/1084 [==============================] - 139s 128ms/step - loss: 0.4699 - accuracy: 0.7670 - val_loss: 0.4968 - val_accuracy: 0.7550
Epoch 6/16
1084/1084 [==============================] - 139s 129ms/step - loss: 0.4353 - accuracy: 0.7920 - val_loss: 0.4756 - val_accuracy: 0.7674
Epoch 7/16
1084/1084 [==============================] - 141s 130ms/step - loss: 0.3971 - accuracy: 0.8157 - val_

In [ ]:
test_domains = ["https://www.google.com", "https://www.quora.com/How-do-I-calculate-the-entropy-of-words", "https://www.sdjfsid983r8ff328h.com/"]
test_domains_df = pd.DataFrame({"domain" : test_domains})
test_domains_df.domain = test_domains_df.domain.map(lambda d: extract(d).domain)
test_domains_df.domain = test_domains_df.domain.map(lambda d : [domain_char_list.index(c) + 1 for c in d])
test_domains_X = pad_sequences(test_domains_df.domain, padding='post')
test_domains_df["predictions"] = model.predict(test_domains_X)
test_domains_df["domain"] = test_domains
test_domains_df.head()

,domain,predictions
0,https://www.google.com,0.954598
1,https://www.quora.com/How-do-I-calculate-the-e...,0.936769
2,https://www.sdjfsid983r8ff328h.com/,0.008544
